In [1]:

uadatapath="hdfs:///user/swethakolalapudi/audio/user_artist_data.txt"
rawUserArtistData = sc.textFile(uadatapath)

In [2]:
rawUserArtistData.take(10)

[u'1000002 1 55',
 u'1000002 1000006 33',
 u'1000002 1000007 8',
 u'1000002 1000009 144',
 u'1000002 1000010 314',
 u'1000002 1000013 8',
 u'1000002 1000014 42',
 u'1000002 1000017 69',
 u'1000002 1000024 329',
 u'1000002 1000025 1']

In [3]:
rawUserArtistData.map(lambda x:float(x.split(" ")[2])).stats()

(count: 24296858, mean: 15.2957624809, stdev: 153.915321302, max: 439771.0, min: 1.0)

In [4]:
from pyspark.mllib.recommendation import Rating,ALS

In [5]:
uaData=rawUserArtistData\
    .map(lambda x:x.split(" "))\
    .filter(lambda x: float(x[2])>=20)\
    .map(lambda x:Rating(x[0],x[1],x[2]))
uaData.persist()

PythonRDD[4] at RDD at PythonRDD.scala:43

In [6]:
uaData.take(10)

[Rating(user=1000002, product=1, rating=55.0),
 Rating(user=1000002, product=1000006, rating=33.0),
 Rating(user=1000002, product=1000009, rating=144.0),
 Rating(user=1000002, product=1000010, rating=314.0),
 Rating(user=1000002, product=1000014, rating=42.0),
 Rating(user=1000002, product=1000017, rating=69.0),
 Rating(user=1000002, product=1000024, rating=329.0),
 Rating(user=1000002, product=1000031, rating=47.0),
 Rating(user=1000002, product=1000055, rating=25.0),
 Rating(user=1000002, product=1000062, rating=71.0)]

In [7]:
model=ALS.trainImplicit(uaData,10,5,0.01)

In [9]:
user = 1000002

In [10]:
recommendations=model.recommendProducts(user,5)

In [11]:
recommendations

[Rating(user=1000002, product=1000113, rating=1.0437801787633956),
 Rating(user=1000002, product=82, rating=0.9607146923882004),
 Rating(user=1000002, product=1205, rating=0.9428474092426006),
 Rating(user=1000002, product=1274, rating=0.9334984230868388),
 Rating(user=1000002, product=1394, rating=0.8537889582986908)]

In [13]:
artistsPath="hdfs:///user/swethakolalapudi/audio/artist_data.txt"
artistLookup=sc.textFile(artistsPath).map(lambda x:x.split("\t"))
artistLookup.persist()

PythonRDD[139] at RDD at PythonRDD.scala:43

In [14]:
userArtists=rawUserArtistData\
    .map(lambda x:x.split(" "))\
    .filter(lambda x:int(x[0])==user and int(x[2])>50)\
    .map(lambda x:x[1]).collect()

In [15]:
for artist in userArtists: 
    print artistLookup.lookup(artist)

[u'Portishead']
[u'A Perfect Circle']
[u'Aerosmith']
[u'Judas Priest']
[u'Metallica']
[u'Foo Fighters']
[u'Counting Crows']
[u'Creed']
[u'Audioslave']
[u'Muse']
[u'(hed) Planet Earth']
[u'Dire Straits']
[u'Free']
[u"Fun Lovin' Criminals"]
[u"Guns N' Roses"]
[u'Satriani, Joe']
[u'A']
[u'Joe Satriani']
[u'Bruce Springsteen']
[u'Goo Goo Dolls']
[u'Fugees']
[u'Michael Jackson']
[u'Roachford']
[u'Barenaked Ladies']
[u'Buckcherry']
[u'Jools Holland']
[u'The Classic Chill Out Album']
[u'Frankie Goes To Hollywood']
[u"King's X"]
[u'Mr. Big']
[u'Dave Weckl']
[u'Dan Reed Network']
[u'Liquid Tension Experiment']
[u'Level 42']
[u'Rage Against the Machine']
[u'Badly Drawn Boy']
[u'Beth Orton']
[u'Dido']
[u'Lenny Kravitz']
[u'Everclear']
[u'Feeder']
[u'Jimi Hendrix']
[u'Red Hot Chili Peppers']
[u'R.E.M.']
[u'Desert Sessions']
[u'The Kleptones']
[u'Jamiroquai']
[u'Led Zeppelin']
[u'Marcus Miller']
[u'Moby']
[u'Miles Davis']
[u'Electric Wizard']
[u'Matchbox Twenty']
[u'The Police']
[u'Nina Simone']
[u

In [ ]:
for rating in recommendations: 
    print artistLookup.lookup(str(rating.product))